Осуществим подключение к бд

In [21]:
%load_ext sql
%sql sqlite:///lab_2.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @lab_2.db'

In [ ]:
Создадим таблицу студентов, групп, городов и факультетов

In [30]:
%%sql 
drop table if exists Students;

CREATE TABLE Students 
(
    student_id integer primary key, 
    Name varchar(50),
    SecondName varchar(50), 
    Faculty_id integer,
    city_id integer,
    group_id integer
    GPA number
);

drop table if exists Faculties;
CREATE TABLE Faculties
(
id integer primary key,
name varchar(50)
);

drop table if exists Groups; 
Create table Groups
(
id integer primary key,
name varchar(50)
);

drop table if exists Cities; 
Create table Cities
(
id integer primary key,
name varchar(50)
);



 * sqlite:///lab_2.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Задание 1 
Перепишите создание таблиц с учетом наличия внешних ключей

In [31]:
%%sql
PRAGMA foreign_keys = ON

 * sqlite:///lab_2.db
Done.


[]

In [32]:
%%sql 

drop table if exists Students;
drop table if exists Faculties;
CREATE TABLE Faculties
(
id integer primary key,
name varchar(50)
);

drop table if exists Groups; 
Create table Groups
(
id integer primary key,
name varchar(50)
);

drop table if exists Cities; 
Create table Cities
(
id integer primary key,
name varchar(50)
);

CREATE TABLE Students 
(
    student_id integer primary key, 
    Name varchar(50),
    SecondName varchar(50), 
    Faculty_id integer,
    City_id integer,
    group_id integer,
    GPA number,
    foreign key(Faculty_id) references Faculties(id),
    foreign key (city_id) references cities(id),
    foreign key (group_id) references groups(id)
    on update cascade
    on delete cascade
);



 * sqlite:///lab_2.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Задание 2
Осуществить вставку во все таблицы с учетом правил на внешние ключи

In [33]:
%%sql
insert into Faculties(id, name) VALUES (1, 'KIB');
insert into Faculties(id, name) VALUES (2, 'IT');
insert into Faculties(id, name) VALUES (3, 'Economics');
insert into Groups(id, name) VALUES (1, '101');
insert into Groups(id, name) VALUES (2, '201');
insert into Groups(id, name) VALUES (3, '202');
insert into Groups(id, name) VALUES (4, '301');
insert into Cities(id, name) VALUES (1, 'Moscow');
insert into Cities(id, name) VALUES (2, 'Voronezh');
insert into Students(student_id, Name, SecondName, Faculty_id, city_id, group_id, GPA) VALUES (1, 'Anton', 'Sidorov', 1, 1, 1, 3.5);
insert into Students(student_id, Name, SecondName, Faculty_id, city_id, group_id, GPA) VALUES (2, 'Alexey', 'Petrov', 2, 2, 2, 3.3);
insert into Students(student_id, Name, SecondName, Faculty_id, city_id, group_id, GPA) VALUES (3, 'Ivan', 'Meshkov', 1, 2, 1, 4.5);
insert into Students(student_id, Name, SecondName, Faculty_id, city_id, group_id, GPA) VALUES (4, 'Elena', 'Kvitova', 3, 1, 4, 4.8);
insert into Students(student_id, Name, SecondName, Faculty_id, city_id, group_id, GPA) VALUES (5, 'Darya', 'Salnikova', 2, 2, 3, 3.4);
insert into Students(student_id, Name, SecondName, Faculty_id, city_id, group_id, GPA) VALUES (6, 'Boris', 'Ivanov', 1, 2, 1, 4);
insert into Students(student_id, Name, SecondName, Faculty_id, city_id, group_id, GPA) VALUES (7, 'Alexander', 'Kapustin', 1, 2, 1, 3.9);

 * sqlite:///lab_2.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## Задание 3
Вывести Имя Студента, Название его факультета, группы и города проживания

In [25]:
%%sql
select Students.name, faculties.name as Faculty, groups.name as group_number, Cities.name as City
from students, faculties, groups, cities
where Students.Faculty_id = faculties.id and Students.group_id = groups.id and Students.City_id = cities.id;

 * sqlite:///lab_2.db
Done.


Name,Faculty,group_number,City
Anton,KIB,101,Moscow
Alexey,IT,201,Voronezh
Ivan,KIB,101,Voronezh
Elena,Economics,301,Moscow
Darya,IT,202,Voronezh
Boris,KIB,101,Moscow
Alexander,KIB,101,Moscow


## Задание 4
Вывести имена всех студентов, обучающихся на факультет IT из Воронежа.

In [35]:
%%sql
select Students.name
from students, faculties, cities
where cities.name = 'Voronezh' and faculties.name = 'IT' and Students.Faculty_id = faculties.id and Students.City_id = cities.id;

 * sqlite:///lab_2.db
Done.


Name
Alexey
Darya


## Задание 5
Для всевозможных пар (Факультет, Город) вывести количество студентов, соответствующее данной группе

In [27]:
%%sql
select faculties.name as faculty, cities.name as city, count(students.student_id) as amt
from students, faculties, cities
where students.faculty_id = cities.id and students.city_id = cities.id
group by faculties.name, cities.name

 * sqlite:///lab_2.db
Done.


faculty,city,amt
Economics,Moscow,3
Economics,Voronezh,2
IT,Moscow,3
IT,Voronezh,2
KIB,Moscow,3
KIB,Voronezh,2


## Задание 6 
Для всех групп вывести средний балл по группе для студентов, не проживающих в Москве, при условии, что количество таких студентов не менее 3 

In [34]:
%%sql
select group_number, avg_gpa
from
(select groups.name as group_number, avg(GPA) as avg_gpa, count(students.student_id) as count
from students, cities, groups
where cities.name != 'Moscow' and students.group_id = groups.id and students.city_id = cities.id
group by groups.name)t
where count >= 3 

 * sqlite:///lab_2.db
Done.


group_number,avg_gpa
101,4.133333333333334
